In [14]:
import pandas as pd
import os

### Prepare data for analys

In [15]:
data = pd.read_csv(filepath_or_buffer='/home/kharlamov/Документы/Project/output/ListProteins.csv')
data = data[data.columns[1:]]

In [16]:
data['sum'] = data.loc[:,"freq_A":"freq_X"].sum(axis=1)
data.loc[:,"freq_A":"freq_X"] = data.loc[:,"freq_A":"freq_X"].div(data["sum"], axis=0)
data = data[data.columns[:-1]]

In [17]:
data.to_csv(path_or_buf='/home/kharlamov/Документы/Project/data/ListProteinsFreq.csv')

In [18]:
data.rename(columns={name : name.split('_')[1] for name in list(data.columns[4:])}, inplace = True)

In [19]:
data_tmp = data[data.columns[3:]]
data_tmp[data_tmp.columns[0]] = map(lambda x: 0 if x == 'human' else 1, data_tmp[data_tmp.columns[0]])
data_tmp = data_tmp[data_tmp.columns[:-1]]

/home/kharlamov/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Запуск линейной регрессии и все такое

In [20]:
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import PolynomialFeatures

In [21]:
data_train, data_test = train_test_split(data_tmp, test_size=0.2)

In [22]:
data_train_x, data_train_y = data_train[data_train.columns[1:]], data_train[data_train.columns[0]]
data_test_x, data_test_y = data_test[data_train.columns[1:]], data_test[data_train.columns[0]]

#### RandomForest

In [23]:
clf = RandomForestClassifier(n_estimators = 100, random_state=14883371)

In [24]:
clf.fit(data_train_x, data_train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=14883371, verbose=0,
            warm_start=False)

In [14]:
print('Accuracy on train = ' + str(clf.score(data_train_x, data_train_y)))
print('Accuracyc on test = ' + str(clf.score(data_test_x, data_test_y))) # Перебрал по сетке, базовые параметры практически выдали лучший скор

Accuracy on train = 1.0
Accuracyc on test = 0.9015954950727358


In [15]:
clf.feature_importances_ # Видно, что в среднем все аминокислоты дают равный вклад

array([1.09013797e-01, 2.52556753e-06, 1.04615363e-01, 4.08605539e-02,
       3.65720353e-02, 2.79032144e-02, 2.92766662e-02, 2.84740732e-02,
       1.14598686e-01, 0.00000000e+00, 5.36887165e-02, 2.61447748e-02,
       2.59648938e-02, 3.96903930e-02, 0.00000000e+00, 7.49220856e-02,
       3.52960029e-02, 3.37930115e-02, 9.05467560e-02, 3.07402153e-02,
       1.73831377e-05, 4.21573927e-02, 2.43998734e-02, 3.13215866e-02,
       0.00000000e+00])

In [16]:
y_pred = clf.predict(data_test_x)
print('F1 Score = ' + str(f1_score(y_pred, data_test_y)))

F1 Score = 0.8200154493176551


In [17]:
print('ROC AUC = ' + str(roc_auc_score(y_pred, data_test_y)))

ROC AUC = 0.9042747102871663


### Linear Regression

In [29]:
clf_2 = LogisticRegression(max_iter=10**6)
clf_cv = GridSearchCV(estimator=clf_2, param_grid={'C' : [0.1, 1, 10, 100, 1000, 10000, 100000]})
clf_cv.fit(data_train_x, data_train_y)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10, 100, 1000, 10000, 100000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
clf_cv.score(data_test_x, data_test_y)

0.8491787893007977

In [92]:
data_train, data_test = train_test_split(data_tmp, test_size=0.2)

In [95]:
x, y = data_train_x[1:], data_train[0]
x_test, y_test = data_test[1:], data_test[0]